In [13]:
!pip install datasets

In [14]:
#Встановлення бібліотек та модулів
from transformers import (DistilBertTokenizer, DistilBertForSequenceClassification,
                          RobertaTokenizer, RobertaForSequenceClassification)
from datasets import load_dataset
import torch
from sklearn.metrics import accuracy_score


Виберемо датасет IMDb для роботи і порівняємо роботу кількох попередньо натренованих моделей з https://huggingface.co/

In [15]:
# Завантаження IMDb-датасету
dataset = load_dataset("imdb")
test_data = dataset["test"]

# Проведемо класифікацію на обмеженій множині тестової вибірки (з метою прийнятної швидкості)
texts = test_data["text"][:500]
true_labels = test_data["label"][:500]

##DistilBERT

In [16]:

# Завантажуємо претреновану вбудовану модель
tokenizer = DistilBertTokenizer.from_pretrained("textattack/distilbert-base-uncased-imdb")
model = DistilBertForSequenceClassification.from_pretrained("textattack/distilbert-base-uncased-imdb")

# Токенізація текстів з урахуванням padding і truncation (для підвищення ефективності моделі)
encoded_inputs = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

# Прогнозування моделі (не обчислюємо градієнти, оскільки це стадія прогнозування)
with torch.no_grad():
    outputs = model(**encoded_inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).numpy()

# Обчислення Accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.95


Бачимо, що вбудована і претренована на сеті imdb дає Accuracy у 95%. Як на мене, ключова роль такого результату - це те, що модель тренувалась вже на відповідному датасеті (попередні результати цієї ж моделі з претренованостю на інших датасетах давали погані результати точності). Крім цього, перед застосуванням моделі тексти були приведені до однакового розміру, що покращує роботу моделей.

Для порівняння, подивимось як спрацює інша модель - Roberta

## Roberta

In [25]:

# Завантаження попередньо натренованої моделі RoBERTa
tokenizer = RobertaTokenizer.from_pretrained("textattack/roberta-base-imdb")
model = RobertaForSequenceClassification.from_pretrained("textattack/roberta-base-imdb")

# Токенізація текстів з урахуванням padding і truncation
encoded_inputs = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

# Прогнозування моделі (не обчислюємо градієнти, оскільки це стадія прогнозування)
with torch.no_grad():
    outputs = model(**encoded_inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).numpy() #обираємо клас з найбільшою ймовірністю

# Обчислення Accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.2f}")

Some weights of the model checkpoint at textattack/roberta-base-imdb were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Accuracy: 0.96


Як бачимо, Roberta показала кращу точність (96%)

Ще подивимось на роботу fasttext для відомих цитат (прогнозування)

In [27]:
!pip install fasttext

import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)

In [28]:
model.predict("To be, or not to be, that is the question", k=5)

(('__label__eng_Latn',
  '__label__pcm_Latn',
  '__label__nld_Latn',
  '__label__kor_Hang',
  '__label__hye_Armn'),
 array([1.00001001e+00, 1.00111101e-05, 1.00090001e-05, 1.00053448e-05,
        1.00030938e-05]))

In [29]:
model.predict("Борітеся – поборете, Вам Бог помагає! За вас правда, за вас сила І воля святая!", k=5)

(('__label__ukr_Cyrl',
  '__label__rus_Cyrl',
  '__label__bul_Cyrl',
  '__label__bel_Cyrl',
  '__label__udm_Cyrl'),
 array([9.94292140e-01, 2.96847359e-03, 1.48159289e-03, 1.21812953e-03,
        2.95291429e-05]))

In [30]:
model.predict("Jedzą, piją, lulki palą, tańce, hulanka, swawola", k=5)

(('__label__pol_Latn',
  '__label__est_Latn',
  '__label__fin_Latn',
  '__label__krc_Cyrl',
  '__label__nld_Latn'),
 array([0.86383253, 0.02589944, 0.01973986, 0.01570418, 0.01077766]))

In [31]:
model.predict("Perseverance is a great element of success. If you only knock long enough and loud enough at the gate, you are sure to wake up somebody", k=5)

(('__label__eng_Latn',
  '__label__kor_Hang',
  '__label__nld_Latn',
  '__label__ces_Latn',
  '__label__pcm_Latn'),
 array([9.99992609e-01, 2.34540785e-05, 1.07576670e-05, 1.03862367e-05,
        1.03298626e-05]))

In [32]:
model.predict("Dime con quién andas y te diré quién eres.", k=5)

(('__label__spa_Latn',
  '__label__ast_Latn',
  '__label__oci_Latn',
  '__label__kor_Hang',
  '__label__hun_Latn'),
 array([9.97403026e-01, 2.57770205e-03, 2.86432642e-05, 2.05222168e-05,
        1.77326747e-05]))

Бачимо, що для відомих цитат модель точно ідентифікує їх мову.

Перевіримо випадково вибране речення з сайту

In [34]:
model.predict("Карпати взимку перетворюються на справжню зимову казку, яка приваблює туристів з усіх куточків світу", k=5)

(('__label__ukr_Cyrl',
  '__label__bel_Cyrl',
  '__label__rus_Cyrl',
  '__label__kan_Knda',
  '__label__tgk_Cyrl'),
 array([1.00000787e+00, 1.20486939e-05, 1.01219821e-05, 1.00068437e-05,
        1.00049920e-05]))

Бачимо, що тут теж модель точно визначила мову. Ймовірно, що чим більша кількість слів у реченні, тим легше моделі здійснити вірну ідентифікацію